Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

Set Device

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Hyperparameters

In [3]:
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 2

Load Data

In [4]:
train_dataset = datasets.MNIST(root='./datatset', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True,)
test_dataset = datasets.MNIST(root='./datatset', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True,)

Create Fully Connected Network

In [5]:
class BRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.LSTM = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(in_features=hidden_size*2, out_features=num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)

        out, _ = self.LSTM(x, (h0, c0))
        out = self.fc(out[:, -1, :])

        return out

Initialize the Network

In [6]:
model = BRNN(input_size=input_size,num_classes=num_classes, hidden_size=hidden_size, num_layers=num_layers).to(device)

Loss and Optimizer

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Train the Network

In [9]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for batch_idx ,(data, target) in enumerate(train_loader):
        #Get data to cuda
        data = data.to(device=device).squeeze(1)
        target = target.to(device=device)
        
        #Get to correct shape
        #data = data.reshape(data.shape[0], -1).squeeze(1)

        #forward
        scores = model(data)
        loss = criterion(scores, target)

        #Backward
        optimizer.zero_grad()
        loss.backward()

        #Gradient Descent or Adam Step
        optimizer.step()
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch+1, num_epochs, batch_idx+1, total_step, loss.item()))

Epoch [1/2], Step [1/938], Loss: 2.2990
Epoch [1/2], Step [2/938], Loss: 2.2889
Epoch [1/2], Step [3/938], Loss: 2.3017
Epoch [1/2], Step [4/938], Loss: 2.3101
Epoch [1/2], Step [5/938], Loss: 2.3131
Epoch [1/2], Step [6/938], Loss: 2.2873
Epoch [1/2], Step [7/938], Loss: 2.2958
Epoch [1/2], Step [8/938], Loss: 2.2792
Epoch [1/2], Step [9/938], Loss: 2.2647
Epoch [1/2], Step [10/938], Loss: 2.2546
Epoch [1/2], Step [11/938], Loss: 2.2420
Epoch [1/2], Step [12/938], Loss: 2.1974
Epoch [1/2], Step [13/938], Loss: 2.1841
Epoch [1/2], Step [14/938], Loss: 2.1786
Epoch [1/2], Step [15/938], Loss: 2.1391
Epoch [1/2], Step [16/938], Loss: 1.9737
Epoch [1/2], Step [17/938], Loss: 1.9718
Epoch [1/2], Step [18/938], Loss: 2.1285
Epoch [1/2], Step [19/938], Loss: 1.8510
Epoch [1/2], Step [20/938], Loss: 2.0463
Epoch [1/2], Step [21/938], Loss: 1.9460
Epoch [1/2], Step [22/938], Loss: 1.9797
Epoch [1/2], Step [23/938], Loss: 1.9582
Epoch [1/2], Step [24/938], Loss: 1.7762
Epoch [1/2], Step [25/938

Check Accuracy

In [10]:
def evaluate(loader, model):
    # Check whether this is training or test data based on .train attribute
    if hasattr(loader.dataset, 'train'):
        if loader.dataset.train:
            print("Checking accuracy on training data")
        else:
            print("Checking accuracy on test data")
    else:
        print("Checking accuracy")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            # Reshape if needed (e.g., for BRNN on MNIST)
            x = x.reshape(-1, 28, 28)

            scores = model(x)
            _, pred = scores.max(1)
            num_correct += (pred == y).sum().item()
            num_samples += pred.size(0)

        print(f'Got {num_correct}/{num_samples} with accuracy {100.0 * num_correct / num_samples:.2f}%')

    model.train()


In [11]:
print("Evaluating on test set...")
evaluate(test_loader, model)

print("Evaluating on train set...")
evaluate(train_loader, model)


Evaluating on test set...
Checking accuracy on test data
Got 9793/10000 with accuracy 97.93%
Evaluating on train set...
Checking accuracy on training data
Got 58683/60000 with accuracy 97.81%
